In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary
import tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#define model parameters
args = {
    "NUM_EPOCHS": 90, #from original paper
    "BATCH_SIZE": 128,
    "MOMENTUM": 0.9,
    "LR_DECAY": 0.0005,
    "LR_INIT": 0.01,
    "IMAGE_DIM": 227,
    "NUM_CLASSES": 1000
}

In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return x
    
    

In [6]:

def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    for batch, (X, y) in tqdm.tqdm(enumerate(dataloader)):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))
        correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        
        #print("Factor = ",i," , Learning Rate = ",optimizer.param_groups[0]["lr"])
#         if batch % 128 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    train_acc.append(correct*100)


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    val_acc.append(100*correct)
    return correct


In [7]:
model = AlexNet(num_classes=args["NUM_CLASSES"]).to(device)
optimizer = optim.SGD(model.parameters(), lr=args["LR_INIT"], momentum=args["MOMENTUM"], weight_decay=args["LR_DECAY"])
loss_fn = nn.CrossEntropyLoss()
train_acc=[]
val_acc =[]



In [8]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

summary(model, (3, 227, 227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
            Conv2d-2          [-1, 256, 27, 27]         614,656
            Conv2d-3          [-1, 384, 13, 13]         885,120
            Conv2d-4          [-1, 384, 13, 13]       1,327,488
            Conv2d-5          [-1, 256, 13, 13]         884,992
            Linear-6                 [-1, 4096]      37,752,832
            Linear-7                 [-1, 4096]      16,781,312
            Linear-8                   [-1, 10]          40,970
Total params: 58,322,314
Trainable params: 58,322,314
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.59
Forward/backward pass size (MB): 5.02
Params size (MB): 222.48
Estimated Total Size (MB): 228.09
----------------------------------------------------------------


In [9]:

for t in range(args['NUM_EPOCHS']):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, model, loss_fn, optimizer)
#     print("test loop")
    correct = test_loop(testloader, model, loss_fn)
print("Done!")
    

Epoch 1
-------------------------------


157it [05:57,  2.28s/it]


KeyboardInterrupt: 